In [112]:
import requests
import json
import os
from WSMethods import *
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d
import vxi11
import math
import plotly.express as px
import plotly.graph_objects as go

In [113]:
class wrapper:
    def __init__(self, instrument: vxi11.Instrument):
        self.instr = instrument

    def __enter__(self):
        return self.instr

    def __exit__(self, ex_type, ex_value, trace):
        self.instr.close()

def getWSrange(ws_ip: str):
    """ startFreq < endFreq
    endFreq(short wavelength) and startFreq(long wavelength)
    """
    result = requests.get('http://'+ws_ip+'/waveshaper/devinfo').json()
    startFreq = result['startfreq']
    endFreq   = result['stopfreq']
    return startFreq, endFreq

def setOSArange(osa_ip: str, startFreq, endFreq):
    """startFreq < endFreq
    """
    assert startFreq <= endFreq
    with wrapper(vxi11.Instrument(osa_ip)) as instr:
        instr.write(f':SENSe:WAVelength:STARt {float(endFreq)}THZ') # short wavelength
        instr.write(f':SENSE:WAVELENGTH:STOP {float(startFreq)}THZ') # long wavelength

def getOSA(osa_ip: str, trace_name):
    c = 299792458

    with wrapper(vxi11.Instrument(osa_ip)) as instr:# 真空の光速
        instr.write(':SENSE:CORRECTION:RVELOCITY:MEDIUM VACUUM')

    with wrapper(vxi11.Instrument(osa_ip)) as instr:
        instr.write(':UNIT:X WAVelength')
    # 基本的にどのように設定しようが、波長単位でデータが帰ってくるので関係なし
    

    #光スペアナのx軸（波長）取得
    with wrapper(vxi11.Instrument(osa_ip) ) as instr:
        wavelength = instr.ask(':TRACe:X? '+trace_name)
        wavelength = wavelength.split(',')
        wavelength = [float(ii) for ii in wavelength]
        wavelength = np.array(wavelength)# m

    freq = c/wavelength/1E12 # THz

    #光スペアナのy軸（dBm/nm）取得
    with wrapper(vxi11.Instrument(osa_ip)) as instr:
        pnbw = instr.ask(':CALCULATE:MATH:TRF:PNBW:BAND?')#正規化帯域幅

    with wrapper(vxi11.Instrument(osa_ip)) as instr:
        data = instr.ask(':TRACe:Y:PDENsity? '+trace_name+',0.1nm')
        data = data.split(',')
        data = [float(ii) for ii in data]
        data = np.array(data)

    optWavelength = wavelength
    optFreq = freq
    optSpectrum = data
    
    
    return optWavelength, optFreq, optSpectrum

def synthesisSpectrum(freq1, spectrum1, freq2, spectrum2, interpFreq):
    interpSpectrum1 = interp1d(freq1, spectrum1, fill_value='extrapolate', kind='nearest')
    interpSpectrum2 = interp1d(freq2, spectrum2, fill_value='extrapolate', kind='nearest')
    return interpFreq, interpSpectrum1(interpFreq) + interpSpectrum2(interpFreq)
    

def synthesisPhase(freq1, phase1, freq2, phase2, interpFreq):
    interpPhase1 = interp1d(freq1, phase1, fill_value='extrapolate')
    interpPhase2 = interp1d(freq2, phase2, fill_value='extrapolate')
    return interpFreq, interpPhase1(interpFreq) + interpPhase2(interpFreq)






In [126]:
ws_ip  = '169.254.6.8'
osa_ip = "169.254.6.9"
trace_name = "TRB"
c = 299792458



# ルーチン
startFreq, endFreq = getWSrange(ws_ip)
setOSArange(osa_ip, startFreq, endFreq)
optWavelength, optFreq, optSpectrum = getOSA(osa_ip, trace_name)

# バックグラウンドノイズは無視
optSpectrum = optSpectrum.clip(-86, None)

# ターゲットスペクトラム算出
centerWavelength = 1545.2 # nm
bandWidth = 2.0 # nm -3dBになるところと定義する。この辺は適当に運用
targetSpectrum = -10*((optWavelength*1E9 - centerWavelength)/(bandWidth/2.0/np.sqrt(0.3)))**2
targetWavelength = optWavelength.copy()
targetFreq = optFreq.copy()



#filterFreq = optFreq.copy()#np.linspace(startFreq, endFreq, 500)
filterFreq = np.arange(startFreq, endFreq, 0.001)
filterWavelength = c/filterFreq/1E12
_, filterSpectrum = synthesisSpectrum(optFreq, -optSpectrum, targetFreq, targetSpectrum, filterFreq)



interpFilterSpectrum = interp1d(filterFreq, filterSpectrum, fill_value='extrapolate')
filterAt = interpFilterSpectrum(c/(centerWavelength*1E-9)*1E-12)

filterSpectrum = filterSpectrum - filterAt -2# 5とか

filterPhase = np.zeros(filterSpectrum.shape)
filterPort = np.ones(filterSpectrum.shape)







In [127]:
fig = go.Figure()
go_element = lambda x,y,name: go.Scatter(x=x,y=y,mode='lines',name=name)
fig.add_trace(go_element(optFreq, optSpectrum, 'optSpectrum'))
fig.add_trace(go_element(targetFreq, targetSpectrum, 'targetSpectrum'))
fig.add_trace(go_element(filterFreq, filterSpectrum, 'filterSpectrum'))
fig.update_layout(xaxis_title='Freq', yaxis_title='power')
fig.show()


In [124]:
fig = go.Figure()
go_element = lambda x,y,name: go.Scatter(x=x,y=y,mode='lines',name=name)
fig.add_trace(go_element(optWavelength, optSpectrum, 'optSpectrum'))
fig.add_trace(go_element(targetWavelength, targetSpectrum, 'targetSpectrum'))
fig.add_trace(go_element(filterWavelength, filterSpectrum, 'filterSpectrum'))
fig.update_layout(xaxis_title='Wavelength[m]', yaxis_title='power')
fig.show()

In [128]:
r = uploadProfile(ws_ip, filterFreq, -filterSpectrum, filterPhase, filterPort)